In [1]:
# Imports
import pandas as pd
from pathlib import Path
from datetime import datetime
import xlsxwriter
import json

In [2]:
df_file_import = pd.read_csv(
    Path("./Resources/Task_SE03.csv")
)

In [3]:
display(df_file_import.head(10))

,Link,Task Name,Pipeline Step,Status,Assigned To
0,PIC_0303_303_005,fx,FX,apr,Vlad Tushevskiy
1,PIC_0303_303_005,comp,Comp,fin,Kimberly Martinez
2,PIC_0303_303_005,anim,Animation,apr,Jason Shulman
3,PIC_0303_303_005,light,Lighting/Rendering,apr,"Forrest Arnold, Patricio Harris"
4,PIC_0303_303_005,ltcompFx,Light Comp,apr,"Bruce Coy, David Silva, Rick Ramirez, Vlad Tus..."
5,PIC_0303_303_005,ltcomp,Light Comp,apr,"Forrest Arnold, Patricio Harris"
6,PIC_0303_303_005,ref,NaN,apr,NaN
7,PIC_0303_303_005,camB,NaN,apr,NaN
8,PIC_0303_303_005,exp,Exports,apr,NaN
9,PIC_0303_303_005,fxNebula,FX,apr,Rick Ramirez


In [4]:
df_file_import.rename({"Pipeline Step" : "category", "Assigned To" : "names"}, axis = 1, inplace = True)
df_file_import_clean = df_file_import.drop(columns = ["Link", "Task Name", "Status"])
df_file_import_clean

,category,names
0,FX,Vlad Tushevskiy
1,Comp,Kimberly Martinez
2,Animation,Jason Shulman
3,Lighting/Rendering,"Forrest Arnold, Patricio Harris"
4,Light Comp,"Bruce Coy, David Silva, Rick Ramirez, Vlad Tus..."
...,...,...
3786,Rigging,Andrei Orehov
3787,Texture,Yinglei Yang
3788,Modeling,"Andrei Orehov, Yinglei Yang"
3789,NaN,NaN


In [5]:
unique_names = [val.strip() for sublist in df_file_import_clean.names.dropna().str.split(",").tolist() for val in sublist]
unique_names = [x.upper() for x in unique_names]
display(unique_names)

['VLAD TUSHEVSKIY',
 'KIMBERLY MARTINEZ',
 'JASON SHULMAN',
 'FORREST ARNOLD',
 'PATRICIO HARRIS',
 'BRUCE COY',
 'DAVID SILVA',
 'RICK RAMIREZ',
 'VLAD TUSHEVSKIY',
 'FORREST ARNOLD',
 'PATRICIO HARRIS',
 'RICK RAMIREZ',
 'VLAD TUSHEVSKIY',
 'RICK RAMIREZ',
 'RICK RAMIREZ',
 'RICK RAMIREZ',
 'RICK RAMIREZ',
 'STEVE GRAVES',
 'BRUCE COY',
 'VLAD TUSHEVSKIY',
 'VLAD TUSHEVSKIY',
 'DAVID GUTMAN',
 'YINGLEI YANG',
 'DAVID SILVA',
 'VLAD TUSHEVSKIY',
 'DAVID SILVA',
 'VLAD TUSHEVSKIY',
 'YINGLEI YANG',
 'YINGLEI YANG',
 'PETER HERLEIN',
 'JASON SHULMAN',
 'CHRIS PIERZ',
 'DAVID GUTMAN',
 'STEVE GRAVES',
 'RICK RAMIREZ',
 'VLAD TUSHEVSKIY',
 'CHRIS PIERZ',
 'DAVID GUTMAN',
 'RICK RAMIREZ',
 'STEVE GRAVES',
 'YINGLEI YANG',
 'STEVE GRAVES',
 'BRUCE COY',
 'DAVID SILVA',
 'RICK RAMIREZ',
 'VLAD TUSHEVSKIY',
 'STEVE GRAVES',
 'RICK RAMIREZ',
 'VAL KHARITONASHVILI',
 'RICK RAMIREZ',
 'RICK RAMIREZ',
 'DAVID SILVA',
 'VLAD TUSHEVSKIY',
 'VLAD TUSHEVSKIY',
 'BRUCE COY',
 'VLAD TUSHEVSKIY',
 'BRUC

In [6]:
names_summary = pd.DataFrame(unique_names, columns = ["names"]).value_counts().reset_index().rename(columns = {0:"count"})
names_summary

,names,count
0,RICK RAMIREZ,269
1,YINGLEI YANG,238
2,CHRIS PIERZ,220
3,VLAD TUSHEVSKIY,215
4,WILLIAM MAUER,197
...,...,...
62,KRIS KELLY,1
63,BADNAMED-KIM SYBERG,1
64,KENNEY KIMBLE,1
65,THOMAS MAINE,1


In [7]:
# reading the data from the file
with open("./Resources/artists_title.txt") as f:
    data = f.read()

# reconstructing the data as a dictionary
js = json.loads(data)
  
print(js)


{'RICK RAMIREZ': 'VFX SUPERVISOR', 'RYAN SMOLAREK': 'VFX SUPERVISOR', 'YINGLEI YANG': 'TD LIGHTING', 'VLAD TUSHEVSKIY': 'FX ARTIST', 'CHRIS PIERZ': 'TBD', 'WILLIAM MAUER': 'TBD', 'STEVE GRAVES': 'CG SUPERVISOR', 'ANDREI OREHOV': 'TD RIGGER', 'DAVID GUTMAN': 'TBD', 'DAVID SILVA': 'LEAD COMPOSITOR', 'JONATHAN BLISS': 'TBD', 'JON GOURLEY': 'TBD', 'ROTOMAKER': 'OUTSOURCE', 'JASON SHULMAN': 'LEAD ANIMATOR', 'ALEXEY CHEPRAKOV': 'TBD', 'VAL KHARITONASHVILI': 'TBD', 'BRUCE COY': 'TBD', 'GARY LOPEZ': 'TBD', 'FORREST ARNOLD': 'TBD', 'NICK SINNOTT': 'TBD', 'PATRICIO HARRIS': 'TBD', 'DUSTIN ADAIR': 'TBD', 'PETER HERLEIN': 'TBD', 'ELIZABETH BODNAR': 'COMPOSITOR', 'SLAVA PONOMAREV': 'TBD', 'DANIEL FAZEL': 'TBD', 'JACOB COY': 'TBD', 'JAVIER GALLEGO': 'COMPOSITOR', 'EDITH CLARA': 'TBD', 'TIM SIMON': 'TBD', 'CHRISTOPHER KNIGHT': 'TBD', 'ALEXIS TARAKJIAN': 'COMPOSITOR', 'HOLLY WENGER': 'TBD', 'SEAN COX': 'TBD', 'VIT ZELICH': 'TBD', 'DAVID NELSON': 'TBD', "ERIK O' DONNELL": 'COMPOSITOR', 'RIGEL BOWEN': '

In [8]:
names_summary["Department"] = names_summary["names"].map(js)
names_summary = names_summary[["Department", "names", "count"]]
print(names_summary.to_string())

         Department                names  count
0    VFX SUPERVISOR         RICK RAMIREZ    269
1       TD LIGHTING         YINGLEI YANG    238
2               TBD          CHRIS PIERZ    220
3         FX ARTIST      VLAD TUSHEVSKIY    215
4               TBD        WILLIAM MAUER    197
5     CG SUPERVISOR         STEVE GRAVES    178
6         TD RIGGER        ANDREI OREHOV    150
7               TBD         DAVID GUTMAN    139
8   LEAD COMPOSITOR          DAVID SILVA     98
9               TBD       JONATHAN BLISS     92
10              TBD          JON GOURLEY     73
11        OUTSOURCE            ROTOMAKER     69
12    LEAD ANIMATOR        JASON SHULMAN     60
13              TBD     ALEXEY CHEPRAKOV     56
14   VFX SUPERVISOR        RYAN SMOLAREK     54
15              TBD  VAL KHARITONASHVILI     51
16        OUTSOURCE         VFX TRACKING     42
17              TBD            BRUCE COY     41
18              TBD           GARY LOPEZ     36
19              TBD       FORREST ARNOLD

In [34]:
# reading the data from the file
with open("./Resources/dept_order.txt") as order:
    order_data = order.read()

# reconstructing the data as a dictionary
order_dict = json.loads(order_data)
  
print(order_dict)

{'VFX SUPERVISOR': 1, 'EXECUTIVE PRODUCER': 2, 'VFX PRODUCER': 3, 'COORDINATOR': 4, 'LEAD COMPOSITOR': 5, 'COMPOSITOR': 6, 'CG SUPERVISOR': 7, 'TD LIGHTING': 8, 'MATTE PAINTER': 9, '3D ARTIST': 10, 'FX SUPERVISOR': 11, 'LEAD FX ARTIST': 12, 'FX ARTIST': 13, 'LEAD ANIMATOR': 14, 'ANIMATOR': 15, '3D CAMERA TRACKING SUPERVISOR': 16, '3D CAMERA TRACKING': 17, 'LEAD TD RIGGER': 18, 'TD RIGGER': 19, 'VFX EDITORIAL MANAGER': 20, 'VFX EDITOR': 21, 'PIPELINE SUPERVISOR': 22, 'OUTSOURCE': 23, 'TBD': 24}


In [35]:
names_summary["order"] = names_summary["Department"].map(order_dict)
names_summary = names_summary[["order", "Department", "names", "count"]]
names_summary = names_summary.sort_values("order")
names_summary

,order,Department,names,count
0,1,VFX SUPERVISOR,RICK RAMIREZ,269
14,1,VFX SUPERVISOR,RYAN SMOLAREK,54
8,5,LEAD COMPOSITOR,DAVID SILVA,98
51,6,COMPOSITOR,NICK DODGE,3
50,6,COMPOSITOR,KIMBERLY MARTINEZ,3
...,...,...,...,...
41,24,TBD,TONY CASTRO,5
15,24,TBD,VAL KHARITONASHVILI,51
42,24,TBD,KEN BRILLIANT,5
45,24,TBD,JOSH MOSSOTTI,5


### Project Setup

In [36]:
project = "Picard"

### Excel Formating & Export

Add a table to the excel 

In [37]:
file_name = "./Resources/" + project + "_end_credit.xlsx"
sheet_name = "end_credit"


Create a Excel File and Setup Worksheet Add a title to the excel

In [38]:
# Create an new Excel file and add a worksheet.
workbook = xlsxwriter.Workbook(file_name)
worksheet = workbook.add_worksheet(sheet_name)
worksheet.hide_gridlines(2)

Setup Column Size

In [39]:
# Setup the columns for the workbook.
worksheet.set_column('A:A', 4)
worksheet.set_column('B:D', 8.5)
worksheet.set_column('E:E', 2)
worksheet.set_column('F:H', 8.5)
worksheet.set_column('I:I', 1)
worksheet.set_column('J:J', 8)


0

Add Bold

In [40]:
# Add a bold format to use to highlight cells.
bold = workbook.add_format({'bold': True})

Add Image

In [41]:
# Insert an image.
# worksheet.insert_image('B5', 'logo.png')

Formating

In [42]:
cell_format = workbook.add_format()
cell_format.set_align("center")
cell_format.set_left()
cell_format.set_right()

cell_dept = workbook.add_format()
cell_dept.set_align("right")


Merge

In [43]:
worksheet.merge_range("B2:H2", "GHOST VFX", cell_format)
worksheet.merge_range("B3:H3", "Burbank, CA", cell_format)

0

Write to Worksheet

In [44]:
# Start from the first cell below the headers.
row_dept = 4
col_dept = 3

# Iterate over the data and write it out row by row.
for dept in (names_summary["Department"]):
    worksheet.write(row_dept, col_dept, dept, cell_dept)
    row_dept += 1

In [45]:
# Start from the first cell below the headers.
row_artist = 4
col_artist = 5

# Iterate over the data and write it out row by row.
for artist in (names_summary["names"]):
    worksheet.write(row_artist, col_artist, artist)
    row_artist += 1

In [46]:
# Start from the first cell below the headers.
row_count = 4
col_count = 9

# Iterate over the data and write it out row by row.
for count in (names_summary["count"]):
    worksheet.write(row_count, col_count, count, cell_format)
    row_count += 1

Add a remark to the excel

In [49]:
worksheet.write(
    len(names_summary)+9, 
    0, 
    "For Internal Use Only", 
    workbook.add_format(
        {"bold": True}
    )
)

worksheet.write(
    len(names_summary)+10, 
    0, 
    "Last update time " + datetime.now().strftime('%H:%M') + ".")


0

In [48]:
workbook.close()